### PPO-clip

> ratio 为 1，模型能否更新；

- 当一个优化周期（epoch）开始时，新策略 $π_θ$ 的参数是从旧策略 $π_{θ_{old}}$ 复制过来的。因此，在进行任何梯度更新之前，对于任何状态 $s_t$ 和动作 $a_t$，$π_θ(a_t | s_t)$ 和 $π_{θ_{old}}(a_t | s_t)$ 的值是完全相等的。所以，在第一次计算损失函数时，对于所有数据点，ratio $r_t$ 的值必然为 1。

$$
L^{CLIP}(θ) = E_t [\min( r_t(θ) * A_t, clip(r_t(θ), 1-ε, 1+ε) * A_t ) ]
$$

- $A_t$ 是一个根据旧策略计算出的值，在策略更新时被视为一个常数，那它的梯度不就是0吗？
    - 这里是关键点：PPO的目标函数是一个代理目标（Surrogate Objective）。我们实际上要优化的还是策略的对数概率 $log π_θ(a_t | s_t)$。梯度的计算最终会回归到策略梯度定理的形式。
        - $r_t(θ) = \frac{π_θ(a_t | s_t)}{π_{θ_{old}}(a_t | s_t)}$
            - ${π_{θ_{old}}(a_t | s_t)}$ 是从 torch 计算图上 detach 的，不参与训练和更新；
    - 当 $r_t$ 在 $[1-\epsilon, 1+\epsilon]$ 区间内时（r_t=1 满足此条件），PPO的梯度等价于：
        - $∇_θ L(θ) ≈ ∇_θ [r_t(θ) * A_t] = A_t * ∇_θ r_t(θ)$
        - 而 $∇_θ r_t(θ)$ 正比于 $∇_θ \log π_θ(a_t | s_t)$。所以，最终的梯度大致是：
            - $∇_θ L(θ) ≈ A_t * ∇_θ \log π_θ(a_t | s_t)$
- 这正是**标准策略梯度（Vanilla Policy Gradient）**的形式！